In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [7]:
clf_dict = {'DT': DecisionTreeClassifier() }
data_type = 'N1' 
model = 'DT'
expt_num = 1
num_splits = 4
TS_Fresh_setting = 'testin dat'

In [8]:
folder = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Fitting Hyperparam Test/tuned_region_only_models_and_results/baseline_'
model = list(clf_dict.keys())[0]
save_name = model + '_' + data_type + '_' + TS_Fresh_setting + '_' + 'expt' + str(expt_num) + '_' +  str(num_splits) + '_fold_results_df.pkl'
print('commencing for ... ' + save_name)


#Change load path to the band power time series folder
load_path = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
load_path_bpw = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
load_path_max_freqs = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Gen_New_Features/generated_feats/'

X_bpw = pd.read_hdf(load_path_bpw + data_type +  'six_second_freq_df.h5', key='df', mode='r')
y = pd.read_hdf(load_path + data_type +  '_y.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + data_type +  '_groups.h5', key='df', mode='r')
X_bpw, y , groups = X_bpw.reset_index(drop = True) , y.reset_index(drop = True) , groups.reset_index(drop = True)

X_max_freqs = pd.read_hdf(load_path_max_freqs + data_type +  'six_second_max_freq_stats_df.h5', key='df', mode='r')
X_max_freqs = X_max_freqs.reset_index(drop = True)

X = pd.concat([X_bpw , X_max_freqs], axis = 1)

commencing for ... DT_N1_testin dat_expt1_4_fold_results_df.pkl


In [9]:
#Transform the X into mean of time series wihin dataframe ___
def function(x):
    return x.values.mean()
    
#Construct the static features
static_features_df = X.apply(np.vectorize(function))
X = static_features_df.copy()

In [10]:
X

,Delta_Central,Theta_Central,Alpha_Central,Sigma_Central,Beta_Central,Gamma_Central,TotalAbsPow_Central,Delta_Frontal,Theta_Frontal,Alpha_Frontal,...,Fourth_Max_Value_Right Parietal,Fourth_Max_Freq_Right Parietal,First_Max_Value_Right Temporal,First_Max_Freq_Right Temporal,Second_Max_Value_Right Temporal,Second_Max_Freq_Right Temporal,Third_Max_Value_Right Temporal,Third_Max_Freq_Right Temporal,Fourth_Max_Value_Right Temporal,Fourth_Max_Freq_Right Temporal
0,0.357054,0.271147,0.143579,0.088512,0.107634,0.032074,37.395927,0.271550,0.316040,0.154706,...,0.043969,6.285227,0.097595,5.167045,0.071912,5.823864,0.056144,5.797727,0.046887,5.876136
1,0.362259,0.269899,0.133596,0.081374,0.111757,0.041116,36.319297,0.284251,0.305415,0.143525,...,0.043892,5.634375,0.084520,5.071875,0.066405,5.309375,0.053125,5.323437,0.044225,5.149219
2,0.406854,0.334486,0.152661,0.049355,0.049246,0.007399,43.074216,0.465542,0.285832,0.137671,...,0.055413,6.496094,0.110095,5.159375,0.083354,5.228125,0.062198,5.475781,0.052861,5.734766
3,0.477266,0.243642,0.151653,0.050000,0.065017,0.012422,37.442328,0.472987,0.258954,0.147300,...,0.046506,5.757812,0.112526,5.339062,0.081581,4.509375,0.061158,6.943750,0.050732,5.798438
4,0.493876,0.275883,0.135107,0.040742,0.047632,0.006758,55.338057,0.485292,0.280343,0.145997,...,0.049813,5.222917,0.109943,3.770833,0.073439,4.327083,0.056906,5.118750,0.047929,5.502083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.435299,0.193446,0.209193,0.070649,0.083038,0.008375,57.827542,0.475867,0.183136,0.185886,...,0.054383,5.621314,0.101862,4.755128,0.073982,5.331891,0.058461,5.465705,0.049633,5.769231
67,0.462307,0.119910,0.095504,0.063631,0.185488,0.073161,31.986673,0.500664,0.102587,0.091624,...,0.039014,4.815809,0.085574,2.772426,0.060740,3.566544,0.046200,4.063603,0.038538,4.878676
68,0.317901,0.094915,0.135905,0.059334,0.308116,0.083829,23.883327,0.296572,0.088342,0.174057,...,0.036819,7.808152,0.072820,4.945924,0.054451,5.569293,0.042216,6.450000,0.035275,6.622826
69,0.521954,0.178997,0.116607,0.089777,0.083018,0.009646,78.386870,0.543619,0.177479,0.118368,...,0.047335,4.586250,0.089603,3.102500,0.066867,3.583750,0.053130,3.710000,0.046004,4.117500


In [11]:
#Drop columns where all values are the same
# Find columns where all values are the same
same_value_columns = X.columns[X.nunique() == 1]
# Drop columns with the same values
X = X.drop(columns=same_value_columns)
print(len(X.columns))

195


In [15]:
#Generate a region to features dictionary - this will enable us to run expts regionally as before
regional_features_dict = {}
region_channel_dict = constants.region_to_channel_dict
regions = list(region_channel_dict.keys())
for region in regions:
    region_features = [col for col in X.columns if '_' + region in col]
    regional_features_dict[region] = region_features 

In [17]:
sum = 0 
for key in regional_features_dict.keys():
    sum += len(regional_features_dict[key])
sum

195